In [6]:
import os
import bs4

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain_community.embeddings import HuggingFaceEmbeddings 
from sentence_transformers import SentenceTransformer

URL_TO_SCRAPE = "https://chjm202.github.io/AI_Search_Ads_test/ads_tmp/ads1.html"
MODELSCOPE_ACCESS_TOKEN = "ms-6634b7ff-46bc-4400-b9f0-d475b0890dab" 
MODELSCOPE_BASE_URL = "https://api-inference.modelscope.cn/v1/"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" 
CHROMA_DB_PATH = "./chroma_db_multilingual_minilm" 

print("--- 1. 正在抓取网页内容... ---")

# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
bs4_strainer = bs4.SoupStrainer(['h1', 'h2', 'p'])
loader = WebBaseLoader(
    web_paths=(URL_TO_SCRAPE,),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

print("--- 2. 正在进行文本分块 (Chunking) 和向量化 (Embedding)... ---")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    add_start_index=True
)
splits = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings, 
    persist_directory=CHROMA_DB_PATH
)

print(f"成功创建 {len(splits)} 个文本块并存入向量数据库。")
print("--- 数据处理完成。 ---")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

--- 1. 正在抓取网页内容... ---
--- 2. 正在进行文本分块 (Chunking) 和向量化 (Embedding)... ---
成功创建 2 个文本块并存入向量数据库。
--- 数据处理完成。 ---


In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-Coder-32B-Instruct", 
    base_url=MODELSCOPE_BASE_URL, 
    api_key=MODELSCOPE_ACCESS_TOKEN,
    temperature=0
) 

system_prompt = (
    "你是一位专业的 AI 助理。请根据提供的上下文 (Context) 来回答问题。如果上下文中没有信息，"
    "请说明你无法找到答案。请用中文简洁明了地回答问题，并引用上下文中的关键信息。"
    "\n\nContext: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("--- RAG 问答链构建完成。 ---")

question_a = "请为我推荐一款酒类产品"
print(f"问题 A: {question_a}")

response_a = rag_chain.invoke({"input": question_a})
print("\n--- 回答 A ---")
print(response_a['answer'])
print("----------------\n")

# 删除本地 ChromaDB 文件夹
# import shutil
# shutil.rmtree("./chroma_db")
# print("本地数据库已清理。")

--- RAG 问答链构建完成。 ---
问题 A: 请为我推荐一款酒类产品

--- 回答 A ---
根据提供的上下文，您可能对具有健康益处的药酒感兴趣。泰山药酒是一个推荐的选择，因为它结合了传统中医药理论和现代炮制技术，对改善肾虚症状、提升免疫功能等方面有显著效果。不过，药酒属于药食同源制品，使用时需要在专业指导下根据个人体质选择，孕妇、儿童及酒精过敏者禁用，慢性病患者应咨询医师后使用。
----------------

